In [0]:
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT REPLACE(SPLIT(current_user(), '@')[0], '.', '_');
DECLARE OR REPLACE VARIABLE resourceType STRING DEFAULT 'Encounter';

In [0]:
SET VARIABLE resourceType = :resource_type; 

SELECT resourceType;

In [0]:
USE IDENTIFIER("fhir_workshop." || schema_use);

In [0]:
SELECT current_catalog(), current_schema();

In [0]:
DECLARE OR REPLACE VARIABLE resource_keys STRING;

SET VAR resource_keys = (
  SELECT 
    array_join(collect_list(DISTINCT resource), "', '")
  FROM 
    fhir_resource_schemas
  WHERE 
    resourceType = resourceType
);

select resource_keys;

In [0]:
DECLARE OR REPLACE VARIABLE resource_stmnt STRING;

SET VARIABLE resource_stmnt = "
CREATE OR REFRESH STREAMING TABLE " || resourceType || "
COMMENT 'Parsed FHIR " || resourceType || " Records'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  *
FROM (
  SELECT
    bundle_uuid
    ,fullUrl as "|| resourceType ||"_url
    ,key
    ,value
  FROM 
    STREAM(fhir_resources)
  WHERE 
    resourceType = '"|| resourceType ||"')
  PIVOT (
    first(value) FOR key IN ('" || resource_keys || "')
  );
";

SELECT resource_stmnt;


In [0]:
EXECUTE IMMEDIATE resource_stmnt;